In [ ]:
import pandas as pd

file_path = "C:\\Users\\ASUS MB511\\OneDrive\\Desktop\\PROJECT INTERNSTUDIO\\Prediction.csv"
df = pd.read_csv(file_path)

df.info() 
df.head()
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')
missing_values=df.isnull().sum()
missing_values[missing_values>0]
df[df['TotalCharges'].isnull()]
df.loc[:, 'TotalCharges'] = df['TotalCharges'].fillna(0)
df.isnull().sum().sum()
df['Churn']=df['Churn'].map({'Yes':1,'No':0})
df['SeniorCitizen']=df['SeniorCitizen'].astype(object)
categorical_col=df.select_dtypes(include=['object']).columns.tolist()
categorical_col.remove('customerID')
df_encoded=pd.get_dummies(df,columns=categorical_col,drop_first=True)
df_encoded.head()          
from sklearn.model_selection import train_test_split
X=df_encoded.drop(columns=['customerID','Churn'])
y=df_encoded['Churn']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

X_train.shape,X_test.shape,y_train.shape,y_test.shape             
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

log_reg=LogisticRegression(max_iter=1000,random_state=42)
dtree=DecisionTreeClassifier(random_state=42)
rf=RandomForestClassifier(n_estimators=100,random_state=42)

log_reg.fit(X_train_scaled,y_train)
dtree.fit(X_train,y_train)
rf.fit(X_train,y_train)

log_reg_preds=log_reg.predict(X_test_scaled)
dtree_preds=dtree.predict(X_test)
rf_preds=rf.predict(X_test)

def evaluate_model(y_true,y_pred,model_name):
    return {
        "Model":model_name,
        "Accuracy":accuracy_score(y_true,y_pred),
        "Precision":precision_score(y_true,y_pred),
        "Recall":recall_score(y_true,y_pred),
        "AUC-ROC":roc_auc_score(y_true,y_pred)
    }
results=[
    evaluate_model(y_test,log_reg_preds,"Logistic Regression"),
    evaluate_model(y_test,dtree_preds,"Decision Tree"),
    evaluate_model(y_test,rf_preds,"Random Forest")
]
results_df=pd.DataFrame(results)
results_df
